<a href="https://colab.research.google.com/github/Vinayak-Sharma12/GENAI/blob/main/Xvi_Conditional_Chain(FeedBack_Agent).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CONDITIONAL CHAIN**

### Basically it is classifier like.

###If this happens go to this llm with this prompt

###and Else if this happens to this llm with this prompt

For Example - Create a app which can handle feedback seperately like if the feedback is positive it reply like tha and if negative it reply like that


In [36]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     


### REQUIRED THINGS :
                      1. Pydantic Output Parser -> So that first llm can only output postive or negative (about feedback)
                      2. Runnable Branch -> Helps to create Conditional Logic
                      3. Runnable Lambda->let yous to define if and else

In [1]:
!pip install langchain langchain_groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant',api_key='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')

In [5]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableBranch,RunnableLambda
from typing import Literal

PARSER READY

In [6]:
parser1=StrOutputParser()

In [7]:
class feedback(BaseModel):
  sentiment:Literal['positive','negative']=Field(description='Give the sentiment')

parser2=PydanticOutputParser(pydantic_object=feedback)

PROMPTS

In [24]:
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}",
    input_variables=['text'],
    partial_variables={'format_instruction': parser2.get_format_instructions()}
)


In [25]:
classifier_chain=prompt1|llm|parser2

TESTING PURPOSE

In [26]:
text="""this is the worst phone ever I  have seen"""
result=classifier_chain.invoke(text)
result

feedback(sentiment='negative')

Testing Done

In [27]:
prompt2=PromptTemplate(
    template="Write a appropriate response for positve feedback {text}",
    input_variables=['text']
)
prompt3=PromptTemplate(
    template="Write a appropriate response for negative feedback {text}",
    input_variables=['text']
)

In [28]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | llm | parser1),
    (lambda x:x.sentiment == 'negative', prompt3 | llm | parser1),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [31]:
chain = classifier_chain | branch_chain

print(chain.invoke("This is best phone ever"))


"Thank you so much for your kind words! I'm thrilled to hear that you're happy with the outcome. I appreciate your feedback and will continue to strive for excellence in my work."


### **GET GRAPH**

In [34]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00


In [35]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
